# Interprétabilité des modèles
## 0. Overview

La montée en complexité des modèles de Machine learning et Deep learning introduit bien souvent des modèles avec d'importantes quantités de paramètres, rendant leurs intérprétations complexes.

Pourtant, il est très important d'être capable de comprendre les décisions prises par un modèle. Cela permet avant tout de les optimiser, mais aussi de les rendre plus accessibles pour des pôles plus orientés métiers.

=> **L'objectif de ce notebook est la mise en pratique de l'interprétabilité des modèles sur des cas concrets**

### 0.0 Packages


In [ ]:
import warnings # On enlève les warnings pour la suite du notebook
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from lime import lime_text
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from lime.lime_text import LimeTextExplainer


### 0.1 Données

Pour la suite de ce notebook, nous nous baserons sur 1 jeu de données :
* **ecommerce_reviews** : ce dataset vise à classifier les reviews d'un questionnaire en ligne suivant plusieurs labels

Son importation est disponible ci-dessous. 

In [ ]:
xls = pd.ExcelFile('ecommerce_reviews.xlsx')
df = pd.read_excel(xls, 'Feuil1')
df.head()

#### 0.1.1 Contexte et détail des données

Considérons que ces données sont extraites d'une toute nouvelle entreprise *X*, qui se positionne sur le marché de la grande distribution. Dans sa perspective d'évolution, cette entreprise souhaite suivre les avis de ses clients sur son système de drive. Pour cela elle met à disposition un questionnaire en ligne. L'une des questions (celle qui nous intéresse aujourd'hui), est, pour les clients insatisfaits du site, d'écrire ce qui les a déplu dans leur expérience.

Pour faciliter la lecture des réponses, une personne est chargée de les labelliser manuellement. Les réponses sont donc ainsi classées suivant une dizaine de labels.

Dans le but de ne plus avoir de traitements manuels, cette entreprise souhaite automatiser la tâche.

Les données comportent 2 champs :
* **Cause_insatisfaction** : correspond à la réponse textuelle de la question *"Qu'est ce qui vous a déplu ?"*
* **Label** : correspond à la labellisation manuelle de la réponse

**Exercice 1** : Représenter graphiquement le volume de réponses par label

# 1 - Classification binaire

Pour commencer, nous allons nous ramener à un cadre assez simple, celui de la classification binaire. 

**Exercice 2** : Prenez connaissance du **Label** le plus présent parmi l'ensemble des réponses. Ajoutez au dataframe *df* une variable *Label_binaire* prenant 2 modalités :

* **1** lorsque la variable *Label* correspond au label le plus présent
* **0** pour tous les autres labels

## 1.1 Interprétabilité d'une classification de textes

Nous allons nous intéresser à la notion d'interprétabilité lors d'une classification basée sur des données textuelles. Pour cela, nous nous appuierons sur le package **lime**.

**Exercice 3** : Séparez les données à disposition en 2 échantillons **stratifiés** sur la base de la variable *Label_binaire* (via la fonction *train_test_split*) en définissant une graîne aléatoire (=1234) :
* un échantillon d'apprentissage (2/3 des données)
* un échantillon de validation (1/3 des données)

*NB : Nous noterons X_train, X_test, y_train, y_test nos échantillons d'apprentissage et de validation*

### 1.1.1 Vectorisation du texte, un réel enjeu du NLP

Le NLP est un domaine de la data science relativement complexe. En cause : le fait que les données soient textuelles, ce qui ne permet pas d'appliquer *directement* les algorithmes de machine learning/deep learning standards.

L'un des principaux enjeu est donc ce que l'on appelle la **vectorisation du texte**. En effet, la problématique est globalement de transformer les données textuelles en vecteurs de nombres, de manière à ce quelles soient compréhensibles et intérprétables par des machines. 

De nombreuses méthodes plus ou moins complexes existent à l'heure actuelle, mais cela reste un sujet de recherche constant. En effet, une mauvaise vectorisation impactera directement la performance des modèles utilisés par la suite.

Pour ce notebook nous sommes donc amenés à vectoriser le texte du champ **Cause_insatisfaction**. N'étant pas le principal objectif, nous allons utiliser un **TF-IDF**.

#### 1.1.1.1 TF-IDF

Le **TF-IDF** (acronymes de « Terme Frequency » et « Inverse Document Frequency ») est une méthode de vectorisation du texte facile à comprendre et relativement efficace. Considérons dans notre cas qu'un document est représenté par un tweet. Le but est d'associer à un mot d'un document une valeur. Plus la valeur sera élevée, et plus le mot sera considéré comme "important", ou "porteur de sens" dans le document. Pour pouvoir associer à un mot d'un document une valeur, on va se baser sur les 2 principes suivants :
* **TF** : on compte le nombre d'apparition du mot dans le document, avec comme idée que si le mot apparaît beaucoup, alors il est important
* **IDF** : on s'intéresse au nombre d'apparition du mot dans l'ensemble des documents, avec comme idée que si le mot apparaît dans l'ensemble des documents, alors ce n'est pas un mot clé, et donc son importance est faible

Globalement, un mot est important dans un document si il y est beaucoup représenté, et n'apparaît pas ou peu dans l'ensemble des documents.

Pour formaliser, on pose $x$ un mot et $y$ un document. La valeur $v$ de ce mot dans le document est définie par :

<center> $ v_{x,y} = tf_{x,y} \times log(\frac{N}{df_x})$ </center> 

Avec : 
*  $tf_{x,y}$ : la fréquence du terme $x$ dans le document $y$ 
* $df_x$ : le nombre de documents contenant le terme $x$
* $N$ : le nombre de documents total


#### 1.1.1.2 Quelques traitements préambules 


Il est généralement important d'effectuer des traitements sur le texte avant de le vectoriser. Dans le cadre de ce notebook, nous nous limiterons à :
* Rendre notre texte insensible à la casse en le passant en minuscule
* Enlever les **stopwords**, c'est à dire les mots trop communs comme "le", "la"... pour l'exemple du Français
* Enlever les accents, dans la mesure ou nombreuses sont les réponses sans accents 

L'un des principaux objectifs de ces traitements est de pouvoir regrouper les mots. Pour l'exemple de l'accent, si l'on trouve dans un tweet les mots "même" et "meme", sans avoir enlevé les accents, on considéra que le mot "même" apparaît une seule fois, et inversement.

Pour les stopwords (français), nous utiliserons le package **nltk**.

In [ ]:
stop_words = stopwords.words("french")
stop_words[:10]

**Exercice 4 :** Vectoriser le texte des tweets à l'aide de la fonction **TfidfVectorizer**. On veillera à passer des paramètres de sorte à :
* Enlever les accents
* Passer le texte en minuscule
* Enlever les stop words définis plus haut
* Définir le tokenizer (fonction qui split le texte en mot) par l'expression régulière **r'\w+'** (paramètre *token_pattern*)

*NB : On notera respectivement train_vectors et test_vectors les données X_train et X_test vectorisées*

*NB2 : Pour passer le contenu d'une colonne d'un dataframe du type **int** vert **string**, on peut utiliser le code suivant :
`data.colonne1 = data.colonne1.astype(str)`*

**Exercice 5** : Entraînez ensuite une forêt aléatoire dans un cadre de classification (les paramètres par défaut suffiront pour l'exemple) sur la base de vos données d'apprentissage vectorisées, en indiquant une graine aléatoire (=1234) pour figer les résultats.

*NB : On notera rf le modèle entraîné*

**Exercice 6 :** Stockez les prédictions dans une variable *pred*, et calculer le taux de réponses bien classées

**Exercice 7** : Le taux de réponses bien classées est relativement bon. Calculez la matrice de confusion.

## 1.2 Package lime


Nous allons maintenant nous appuyer sur le package lime pour s'intéresser à l'intérprétabilité du modèle construit. Nous allons essayer de décomposer les prédictions de sorte à obtenir les contributions de chaque mots.

### 1.2.2 Principe


Le but principal du package lime est d'expliquer la prédiction d'un modèle en la décomposant de sorte à ce que même les *non-experts* soient capables de la comprendre et l'intérpréter.
Pour décomposer la prédiction, lime propose de construire un second modèle facilement interprétable, qui permette localement de reproduire le comportement du vrai modèle.

Lime essait donc de proposer un modèle dit *explainer*, regroupant 3 grands principes :
* **Interprétable** : Le modèle doit permettre une compréhension entre l'input et l'output. 
* **Fidélité locale** : Le modèle doit être localement fidèle à la réalité 
* **Agnostique** : L'*explainer* doit pouvoir expliquer n'importe quel modèle sans formuler d'hypothèses dessus


#### 1.2.2.1 Intuition

Intuitivement, lime propose d'approximer localement le comportement d'un modèle construit, et ce, par une fonction linéaire (des arbres de désision sont aussi proposés, mais nous ne nous y intéresserons pas). Approximer un modèle de manière globale est très complexe, c'est pourquoi lime propose de se limiter à l'échelle locale, c'est à dire autour du voisinage d'un point de données.

Pour cela, on choisit le point de données que l'on souhaite expliquer. On notera ce point *instance* par la suite. On perturbe cette instance de manière à obtenir d'autres points de données qui définieront sont voisinage. La figure ci-dessous illustre l'intuition de cette procédure.
<img src="lime.png" alt="Drawing" style="width: 400px;" align="center"/>

Ici, la croix <font color='red'>rouge vif</font> représente l'instance initiale. La fonction de décision du modèle initial est représentée par le fond bleu / rose et est clairement non linéaire. Les points de données générés autour de l'instance sont pondérés en fonction de leur proximité à l'instance (le poids est ici indiqué par la taille des points)

Lime apprend ensuite un modèle linéaire (ligne pointillée) qui se rapproche beaucoup du modèle au voisinage de l'instance, mais pas de manière globale.






### 1.2.2 Fonctionnement théorique

#### 1.2.2.1 Théorie

LIME (Local Interpretable Model-agnostic Explanation) cherche donc à construire une fonction nommée *explainer* qui permette un **compromis entre fidélité et interprétabilité**.
Pour cela, lime essaie de minimiser la quantité suivante : 


<center> $ \xi(x) = \underset{g \in G}{\text{argmin}}\ \mathcal{L}(f,g,\pi_x) + \Omega(g) $ </center> 

Avec : 
* $f$ notre modèle
* $x$ une instance
* $g$ notre modèle explicatif et interprétable (*explainer*). lime propose des modèles linéaires ainsi que des arbres de décisions comme modèle explicatif
* $\pi_x$ une mesure de proximité permettant de définir un *voisinage* autour de $x$

Globalement :
* **le premier terme** mesure l'infidélité de $g$ dans l'approximation de $f$, dans la proximité définie par $\pi$. C'est pourquoi on cherche à minimiser cette quantité, de sorte à ce que la fonction $g$ soit la plus proche possible de $f$. Ce terme est appelé ***locality-aware loss*** dans le papier 

* **Le second terme** mesure la complexité du modèle explicatif $g$. Dans le cas d'un arbre de classification, la complexité serait la profondeur de l'arbre. Dans un cadre de régression, il s'agit du nombre de paramètres non nuls 

#### 1.2.2.2 Quelques notations

Avant de poursuivre, on pose :
* $x'$ un vecteur binaire $\in \{0,1\}^{d}$ (appelé *vecteur interprétable de l'instance* $x$), où $d$ correspond au nombre de variables. Par exemple, dans notre cadre de classification de texte, $d$ correspond au nombre de mots du vocabulaire. Un 1 signifie que le mot apparaît dans $x$, et inversement pour un 0.
* $z'$ : une fraction des éléments non nuls de $x'$. $z'$ correspond donc à un échantillon des $1$ présents dans $x'$.
* $z$ le point de données constitué des variables ou $z'$ possède un 1 (c'est globalement le même principe que pour $x$ : $z'$ correspond au *vecteur interprétable de* $z$)
* $f(z)$ : le label associé par notre modèle
* $g(z')$ : **correspond au modèle explicatif appris par lime**



Afin de garantir à la fois l’interprétabilité et la fidélité, la ***locality-aware loss*** est minimisée tout en maintenant le second terme suffisamment bas pour pouvoir être interprété par *des non experts*.


#### 1.2.2.3 Génération de points de données


Si l'on reprend, nous avons noté $g$ le modèle explicatif à apprendre, $z'$ une instance générée du jeu d'apprentissage et $f(z)$ son label prédit par le modèle $f$.

Afin de générer un ensemble complet de points de données pour l'apprentissage de $g$, lime effectue un échantillonnage aléatoire à partir de $x’$. En d’autres termes, lime créé plusieurs $z’$ à partir d’une seule instance $x$. 

Ces nouveaux points de données sont ensuite pondérés par $\pi_x$ pour se concentrer davantage sur les points générés $z$ qui sont les plus proches de $x$. On a donc à disposition des points générés, et leur label associé par la fonction $f$.

Sachant cela, l'équation vue au dessus est optimisée pour apprendre le modèle explicatif $g$. Lime ne dépend donc pas du type de modèle d'origine pour fournir des *explications* (modèle explicatif indépendant du modèle original).

#### 1.2.2.4 Intérprétation linéaire

Posons :
* $g(z') = w . z'$ (le modèle explicatif linéaire, avec $w$ le vecteur de paramètres, i.e poids)
* $\pi_x(z)$ un noyau exponentiel : $\text{exp}(\frac{-D(x,z)^2}{\sigma^2})$, où $D$ est une fonction de distance (distance cosinus pour du texte)
* On a la ***locality-aware loss*** qui correspond à la ***square loss*** :

<center> $ \mathcal{L}(f,g,\pi_x) = \sum\limits_{z,z'\in \mathcal{Z}}\pi_x(z)\ (f(z) - g(z'))^2$ </center> 

L'algorithme est finalement le suivant :

<img src="lime2.png" alt="Drawing" style="width: 600px;" align="center"/>


$K$ est une limite sur le nombre de variables à conserver dans le modèle explicatif. Par exemple, pour le texte, $K$ indique le nombre de mots à prendre en compte. De manière à ne pas conserver "trop" de paramètres, lime fait en sorte que $\Omega$ tende à l’infini si la taille du vecteur $w \ge K$. 

Au final, ce sont les poids appris ($w$) par le modèle explicatif $g$ qui sont la sortie de l'algorithme.



#### 1.2.2.5 Retour au problème

La forêt construite plus haut provient du package *sklearn*, et s'appuie sur la représentation vectorisée des textes. Cela rend l'intérprétation complexe. C'est pourquoi nous allons nous baser sur le package **lime**.

Dans un premier temps, nous allons construire un *pipeline* d'opération via la fonction *make_pipeline* du package *sklearn*, composé de notre TF-IDF et de notre forêt.

In [ ]:
c = make_pipeline(vectorizer, rf)
c

Nous allons ensuite choisir un point de données du jeu de test, et essayer de décomposer la prédiction obtenue.

In [ ]:
indice = 122
print("Réponse correspondant au point choisi : {}\nProbabilité d'appartenance à la classe 0 (=labels Autres) : {}\nProbabilité d'appartenance à la classe 1 (=label Performance) : {}"
      .format(X_test.iloc[indice],c.predict_proba([X_test.iloc[indice]])[0][0],c.predict_proba([X_test.iloc[indice]])[0][1]))

Pour la réponse choisie, le modèle tend à prédire le label *Performance* à hauteur d'environ 78%. Nous allons maintenant essayer de décomposer cette prédiction.

Le packege **lime** propose de décomposer les prédictions en classification pour bon nombre de modèles, et pour de nombreux cadres, comme la classification de textes, d'images etc.

Pour notre classification de texte, nous allons utiliser la classe **LimeTextExplainer**, qui prend en paramètre la liste des labels présents.

In [ ]:
nom_labels = ["Autres","Performance"]
explainer = LimeTextExplainer(class_names=nom_labels)

Cette classe dispose d'une méthode **explain_instance** qui permet de décomposer la prédiction pour un point de données. Pour cela, on lui passe en paramètre le texte de la réponse concernée, les probabilités prédites pour les différentes classes, ainsi que le nombre de features explicatives que nous voulons avoir en sortie (paramètre *num_features*, qui correspond au $K$ de l'algorithme).

Plus précisément, si l'on passe $n$ en valeur au paramètre *num_features*, nous récupérerons les $n$ mots les plus importants avec leur contributions dans la prédiction.

In [ ]:
exp = explainer.explain_instance(X_test.iloc[indice], c.predict_proba, num_features=10)
print('Indice du tweet : {}'.format(indice))
print('Probabilité du label Performance =', c.predict_proba([X_test.iloc[indice]])[0,1])
print('Vraie label: {}'.format(nom_labels[y_test.iloc[indice]]))

On peut donc observer le contenu de la variable exp via la méthode *as_list*.

In [ ]:
exp.as_list()

L'interprétation est relativement simple. Le label prédit est *Performance* à environ 78%. On constate que le mot *lent* contribue à augmenter cette probabilité de 0.75.

### 1.2.1 Visualisation

Le package lime propose aussi des visualisation intéressantes des résultats. Dans notre cadre de classification binaire, le graphique du dessous, toujours pour notre point de données exemple, représente en <font color='green'>vert</font> les mots ayant le plus contribués à prédire le label *Performance*, et en <font color='red'>rouge</font> le label *Autres*.

In [ ]:
%matplotlib inline
fig = exp.as_pyplot_figure()

Une autre méthode de la classe **LimeTextExplainer** permet des visualisation plus avancées, allant jusqu'à mettre en relief les mots les plus importants dans la réponse.

In [ ]:
exp.show_in_notebook(text=False)

In [ ]:
exp.show_in_notebook(text=True)

# 2. Classification multi-classes

**Exercice Bilan** : Reparter des données initiales, et ré-appliquer le même procédé, mais dans une classification multiclasses.

# Bibliographie

Papier du package lime : https://arxiv.org/pdf/1602.04938.pdf